In [3]:
from diffusers import StableDiffusionPipeline
import torch

pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

unet/diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
!pip -q install diffusers==0.30.0 transformers accelerate safetensors controlnet-aux==0.0.7 opencv-python pillow gradio==4.44.1 xformers --extra-index-url https://download.pytorch.org/whl/cu121

In [1]:

from __future__ import annotations
import os
import io
import math
import argparse
from dataclasses import dataclass
from typing import Optional, Tuple, List

import torch
from PIL import Image, ImageOps
import numpy as np

In [2]:

from diffusers import (
    StableDiffusionPipeline,
    StableDiffusionInpaintPipeline,
    StableDiffusionControlNetPipeline,
    ControlNetModel,
)


In [3]:
# ControlNet pre/post utils (canny, openpose, depth, segmentation)
try:
    from controlnet_aux import OpenposeDetector, HEDdetector, MidasDetector, MLSDdetector
    from controlnet_aux.util import HWC3
except Exception:
    OpenposeDetector = None
    HEDdetector = None
    MidasDetector = None
    MLSDdetector = None


In [4]:
# ------------------------------
# Config
# ------------------------------

BASE_SD15 = "runwayml/stable-diffusion-v1-5"
INPAINT_SD15 = "runwayml/stable-diffusion-inpainting"
CN_CANNY = "lllyasviel/sd-controlnet-canny"
CN_OPENPOSE = "lllyasviel/sd-controlnet-openpose"
CN_DEPTH = "lllyasviel/sd-controlnet-depth"
CN_SEG = "lllyasviel/sd-controlnet-seg"

OUTPUT_DIR = "outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [5]:
# ------------------------------
# Utilities
# ------------------------------

def to_pil(img: Image.Image | np.ndarray) -> Image.Image:
    if isinstance(img, Image.Image):
        return img.convert("RGB")
    arr = img
    if arr.dtype != np.uint8:
        arr = np.clip(arr, 0, 255).astype(np.uint8)
    return Image.fromarray(arr).convert("RGB")


def load_mask(mask_img: Image.Image | np.ndarray, size: Tuple[int, int]) -> Image.Image:
    """Load/resize to binary/soft mask in L mode (0-255). White = paint/edit region."""
    m = to_pil(mask_img).convert("L").resize(size, Image.BILINEAR)
    return m


def ensure_device(lowvram: bool = True) -> Tuple[torch.device, torch.dtype]:
    if torch.cuda.is_available():
        device = torch.device("cuda")
        dtype = torch.float16 if lowvram else torch.float32
    elif torch.backends.mps.is_available():
        device = torch.device("mps")
        dtype = torch.float16 if lowvram else torch.float32
    else:
        device = torch.device("cpu")
        dtype = torch.float32
    return device, dtype


In [6]:
# ------------------------------
# Loaders
# ------------------------------

def load_base_pipeline(device: torch.device, dtype: torch.dtype, lowvram: bool = True) -> StableDiffusionPipeline:
    pipe = StableDiffusionPipeline.from_pretrained(
        BASE_SD15, torch_dtype=dtype, safety_checker=None
    )
    pipe = pipe.to(device)
    if lowvram:
        pipe.enable_attention_slicing()
        pipe.enable_vae_slicing()
        try:
            pipe.enable_xformers_memory_efficient_attention()
        except Exception:
            pass
    return pipe


def load_inpaint_pipeline(device: torch.device, dtype: torch.dtype, lowvram: bool = True) -> StableDiffusionInpaintPipeline:
    ipipe = StableDiffusionInpaintPipeline.from_pretrained(
        INPAINT_SD15, torch_dtype=dtype, safety_checker=None
    )
    ipipe = ipipe.to(device)
    if lowvram:
        ipipe.enable_attention_slicing()
        ipipe.enable_vae_slicing()
        try:
            ipipe.enable_xformers_memory_efficient_attention()
        except Exception:
            pass
    return ipipe


def load_controlnet_pipeline(
    controlnet_ids: List[str], device: torch.device, dtype: torch.dtype, lowvram: bool = True
) -> StableDiffusionControlNetPipeline:
    cns = [ControlNetModel.from_pretrained(cid, torch_dtype=dtype) for cid in controlnet_ids]
    cpipe = StableDiffusionControlNetPipeline.from_pretrained(
        BASE_SD15, controlnet=cns if len(cns) > 1 else cns[0], torch_dtype=dtype, safety_checker=None
    )
    cpipe = cpipe.to(device)
    if lowvram:
        cpipe.enable_attention_slicing()
        cpipe.enable_vae_slicing()
        try:
            cpipe.enable_xformers_memory_efficient_attention()
        except Exception:
            pass
    return cpipe



In [7]:
# ------------------------------
# Control image builders
# ------------------------------

def canny_from_image(img: Image.Image, low: int = 100, high: int = 200) -> Image.Image:
    import cv2
    arr = np.array(img.convert("RGB"))
    edges = cv2.Canny(arr, low, high)
    edges = np.stack([edges] * 3, axis=-1)
    return Image.fromarray(edges)


def openpose_from_image(img: Image.Image) -> Image.Image:
    if OpenposeDetector is None:
        raise RuntimeError("controlnet-aux not installed with OpenposeDetector")
    det = OpenposeDetector.from_pretrained("lllyasviel/ControlNet")
    pose = det(img)
    return to_pil(pose)


def depth_from_image(img: Image.Image) -> Image.Image:
    if MidasDetector is None:
        raise RuntimeError("controlnet-aux not installed with MidasDetector")
    det = MidasDetector.from_pretrained("lllyasviel/ControlNet")
    depth = det(img)
    return to_pil(depth)


In [8]:
# ------------------------------
# Generation stages
# ------------------------------

def stage_background(
    pipe: StableDiffusionPipeline,
    prompt: str,
    negative_prompt: str = "",
    seed: Optional[int] = 42,
    width: int = 768,
    height: int = 512,
    guidance_scale: float = 7.0,
    steps: int = 30,
) -> Image.Image:
    generator = torch.Generator(device=pipe.device).manual_seed(seed) if seed is not None else None
    img = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        width=width,
        height=height,
        guidance_scale=guidance_scale,
        num_inference_steps=steps,
        generator=generator,
    ).images[0]
    return img


def stage_inpaint(
    ipipe: StableDiffusionInpaintPipeline,
    base_img: Image.Image,
    mask_img: Image.Image,
    prompt: str,
    negative_prompt: str = "",
    seed: Optional[int] = 123,
    strength: float = 0.85,
    guidance_scale: float = 7.5,
    steps: int = 30,
) -> Image.Image:
    mask = load_mask(mask_img, base_img.size)
    generator = torch.Generator(device=ipipe.device).manual_seed(seed) if seed is not None else None
    out = ipipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        image=base_img,
        mask_image=mask,
        guidance_scale=guidance_scale,
        num_inference_steps=steps,
        strength=strength,
        generator=generator,
    ).images[0]
    return out


def stage_controlnet(
    cpipe: StableDiffusionControlNetPipeline,
    prompt: str,
    control_images: List[Image.Image] | Image.Image,
    negative_prompt: str = "",
    seed: Optional[int] = 7,
    steps: int = 25,
    guidance_scale: float = 7.5,
    conditioning_scale: float | List[float] = 1.0,
    width: Optional[int] = None,
    height: Optional[int] = None,
    init_image: Optional[Image.Image] = None,
    strength: float = 0.55,
) -> Image.Image:
    generator = torch.Generator(device=cpipe.device).manual_seed(seed) if seed is not None else None

    # text2img vs img2img
    if init_image is None:
        out = cpipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            image=control_images,
            num_inference_steps=steps,
            guidance_scale=guidance_scale,
            controlnet_conditioning_scale=conditioning_scale,
            width=width,
            height=height,
            generator=generator,
        ).images[0]
    else:
        out = cpipe.img2img(
            prompt=prompt,
            negative_prompt=negative_prompt,
            image=init_image,
            control_image=control_images,
            num_inference_steps=steps,
            guidance_scale=guidance_scale,
            controlnet_conditioning_scale=conditioning_scale,
            strength=strength,
            generator=generator,
        ).images[0]
    return out


In [9]:
# ------------------------------
# LoRA helpers (optional)
# ------------------------------

def maybe_load_lora(pipe_or_cpipe, lora_repo_or_path: Optional[str], weight: float = 0.75):
    if not lora_repo_or_path:
        return pipe_or_cpipe
    try:
        pipe_or_cpipe.load_lora_weights(lora_repo_or_path)
        pipe_or_cpipe.fuse_lora(lora_scale=weight)
        print(f"[LoRA] Loaded and fused from {lora_repo_or_path} with scale {weight}")
    except Exception as e:
        print(f"[LoRA] Failed to load: {e}")
    return pipe_or_cpipe


In [10]:
# ------------------------------
# Example end-to-end scripted pipeline
# ------------------------------

def run_scripted_example(lowvram: bool = True):
    device, dtype = ensure_device(lowvram)
    print(f"Device: {device}, dtype: {dtype}")

    pipe = load_base_pipeline(device, dtype, lowvram)
    pipe = maybe_load_lora(pipe, None)

    bg_prompt = (
        "cinematic sunset sky over a tranquil lake, soft clouds, golden hour lighting, ultra-detailed landscape"
    )
    background = stage_background(pipe, prompt=bg_prompt, width=832, height=512, steps=28)
    background.save(os.path.join(OUTPUT_DIR, "01_background.png"))


    cpipe = load_controlnet_pipeline([CN_CANNY], device, dtype, lowvram)


    w, h = background.size
    sketch = Image.new("RGB", (w, h), (0, 0, 0))
    import cv2
    sk = np.zeros((h, w, 3), dtype=np.uint8)
    cv2.rectangle(sk, (int(0.58*w), int(0.45*h)), (int(0.83*w), int(0.8*h)), (255, 255, 255), 3)
    sketch = Image.fromarray(sk)
    canny = canny_from_image(sketch, 50, 150)

    cn_prompt = "a cozy wooden lakeside cottage with warm window lights, high detail, photorealistic"
    cn_img = stage_controlnet(
        cpipe,
        prompt=cn_prompt,
        control_images=canny,
        steps=20,
        guidance_scale=8.0,
        conditioning_scale=1.2,
        init_image=background,
        strength=0.55,
    )
    cn_img.save(os.path.join(OUTPUT_DIR, "02_midground_controlnet.png"))


    ipipe = load_inpaint_pipeline(device, dtype, lowvram)

    mask = Image.new("L", (w, h), 0)

    m = np.array(mask)
    cv2.ellipse(m, (int(0.22*w), int(0.78*h)), (int(0.12*w), int(0.06*h)), 0, 0, 360, 255, -1)
    mask = Image.fromarray(m)

    fg_prompt = "a small wooden rowboat floating on the water, gentle ripples, natural reflections, photorealistic"
    composed = stage_inpaint(
        ipipe,
        base_img=cn_img,
        mask_img=mask,
        prompt=fg_prompt,
        strength=0.9,
        steps=28,
    )
    composed.save(os.path.join(OUTPUT_DIR, "03_foreground_inpaint.png"))

    print("Saved example outputs to:", OUTPUT_DIR)



In [11]:
# ------------------------------
# Example end-to-end scripted pipeline
# ------------------------------

def run_scripted_example(lowvram: bool = True):
    device, dtype = ensure_device(lowvram)
    print(f"Device: {device}, dtype: {dtype}")


    pipe = load_base_pipeline(device, dtype, lowvram)
    pipe = maybe_load_lora(pipe, None)

    bg_prompt = (
        "cinematic sunset sky over a tranquil lake, soft clouds, golden hour lighting, ultra-detailed landscape"
    )
    background = stage_background(pipe, prompt=bg_prompt, width=832, height=512, steps=28)
    background.save(os.path.join(OUTPUT_DIR, "01_background.png"))

    cpipe = load_controlnet_pipeline([CN_CANNY], device, dtype, lowvram)

    w, h = background.size
    sketch = Image.new("RGB", (w, h), (0, 0, 0))
    import cv2
    sk = np.zeros((h, w, 3), dtype=np.uint8)
    cv2.rectangle(sk, (int(0.58*w), int(0.45*h)), (int(0.83*w), int(0.8*h)), (255, 255, 255), 3)
    sketch = Image.fromarray(sk)
    canny = canny_from_image(sketch, 50, 150)

    cn_prompt = "a cozy wooden lakeside cottage with warm window lights, high detail, photorealistic"
    cn_img = stage_controlnet(
        cpipe,
        prompt=cn_prompt,
        control_images=canny,
        steps=20,
        guidance_scale=8.0,
        conditioning_scale=1.2,
        init_image=background,
        strength=0.55,
    )
    cn_img.save(os.path.join(OUTPUT_DIR, "02_midground_controlnet.png"))


    ipipe = load_inpaint_pipeline(device, dtype, lowvram)

    mask = Image.new("L", (w, h), 0)

    m = np.array(mask)
    cv2.ellipse(m, (int(0.22*w), int(0.78*h)), (int(0.12*w), int(0.06*h)), 0, 0, 360, 255, -1)
    mask = Image.fromarray(m)

    fg_prompt = "a small wooden rowboat floating on the water, gentle ripples, natural reflections, photorealistic"
    composed = stage_inpaint(
        ipipe,
        base_img=cn_img,
        mask_img=mask,
        prompt=fg_prompt,
        strength=0.9,
        steps=28,
    )
    composed.save(os.path.join(OUTPUT_DIR, "03_foreground_inpaint.png"))

    print("Saved example outputs to:", OUTPUT_DIR)


In [12]:
# ------------------------------
# Gradio demo
# ------------------------------

def build_demo(lowvram: bool = True):
    import gradio as gr

    device, dtype = ensure_device(lowvram)
    pipe = load_base_pipeline(device, dtype, lowvram)
    ipipe = load_inpaint_pipeline(device, dtype, lowvram)


    cn_cache = {}

    def run_background(prompt, negative, width, height, steps, guidance, seed):
        img = stage_background(
            pipe,
            prompt=prompt,
            negative_prompt=negative,
            width=width,
            height=height,
            steps=steps,
            guidance_scale=guidance,
            seed=seed if seed >= 0 else None,
        )
        return img

    def run_controlnet(init_image, control_type, control_image, prompt, negative, steps, guidance, cn_scale, strength, seed):
        if init_image is None:
            return None
        # choose controlnet model
        key = control_type
        if key not in cn_cache:
            if control_type == "canny":
                cpipe = load_controlnet_pipeline([CN_CANNY], device, dtype, lowvram)
            elif control_type == "openpose":
                cpipe = load_controlnet_pipeline([CN_OPENPOSE], device, dtype, lowvram)
            elif control_type == "depth":
                cpipe = load_controlnet_pipeline([CN_DEPTH], device, dtype, lowvram)
            elif control_type == "seg":
                cpipe = load_controlnet_pipeline([CN_SEG], device, dtype, lowvram)
            else:
                raise gr.Error("Unknown control type")
            cn_cache[key] = cpipe
        else:
            cpipe = cn_cache[key]

        if control_image is None:
            # derive control image from init
            if control_type == "canny":
                control_image = canny_from_image(init_image)
            elif control_type == "openpose":
                control_image = openpose_from_image(init_image)
            elif control_type == "depth":
                control_image = depth_from_image(init_image)
            else:
                control_image = canny_from_image(init_image)

        # resize control to init
        control_image = to_pil(control_image).resize(init_image.size, Image.BILINEAR)

        out = stage_controlnet(
            cpipe,
            prompt=prompt,
            negative_prompt=negative,
            control_images=control_image,
            steps=steps,
            guidance_scale=guidance,
            conditioning_scale=cn_scale,
            init_image=init_image,
            strength=strength,
            seed=seed if seed >= 0 else None,
        )
        return out

    def run_inpaint(init_image, mask_image, prompt, negative, steps, guidance, strength, seed):
        if init_image is None or mask_image is None:
            return None
        out = stage_inpaint(
            ipipe,
            base_img=init_image,
            mask_img=mask_image,
            prompt=prompt,
            negative_prompt=negative,
            steps=steps,
            guidance_scale=guidance,
            strength=strength,
            seed=seed if seed >= 0 else None,
        )
        return out

    with gr.Blocks(title="Layered Diffusion (SD 1.5)") as demo:
        gr.Markdown("# Layered Image Generation — SD 1.5 + Inpainting + ControlNet")

        with gr.Tab("1) Background"):
            with gr.Row():
                prompt = gr.Textbox(label="Prompt", value="sunset over mountains, dramatic light, ultrarealistic")
                negative = gr.Textbox(label="Negative Prompt", value="low quality, blurry, deformed")
            with gr.Row():
                width = gr.Slider(512, 1024, value=768, step=64, label="Width")
                height = gr.Slider(512, 1024, value=512, step=64, label="Height")
                steps = gr.Slider(10, 60, value=28, step=1, label="Steps")
                guidance = gr.Slider(1.0, 12.0, value=7.5, step=0.1, label="CFG")
                seed = gr.Number(value=42, precision=0, label="Seed (-1 for random)")
            gen_btn = gr.Button("Generate Background")
            bg_out = gr.Image(type="pil", label="Background")
            gen_btn.click(run_background, [prompt, negative, width, height, steps, guidance, seed], bg_out)

        with gr.Tab("2) ControlNet (optional)"):
            init_image = gr.Image(type="pil", label="Init Image (use Background)")
            with gr.Row():
                control_type = gr.Radio(["canny", "openpose", "depth", "seg"], value="canny", label="Control Type")
                control_image = gr.Image(type="pil", label="Control Image (optional; auto if empty)")
            with gr.Row():
                c_prompt = gr.Textbox(label="Prompt", value="add a modern wooden cabin at the lakeside, photorealistic, high detail")
                c_negative = gr.Textbox(label="Negative", value="lowres, artifacts, bad anatomy")
            with gr.Row():
                c_steps = gr.Slider(10, 60, value=24, step=1, label="Steps")
                c_guidance = gr.Slider(1.0, 12.0, value=8.0, step=0.1, label="CFG")
                cn_scale = gr.Slider(0.1, 2.0, value=1.0, step=0.05, label="CN Scale")
                c_strength = gr.Slider(0.1, 1.0, value=0.55, step=0.05, label="Img2Img Strength")
                c_seed = gr.Number(value=7, precision=0, label="Seed (-1 for random)")
            c_btn = gr.Button("Apply ControlNet (img2img)")
            cn_out = gr.Image(type="pil", label="ControlNet Output")
            c_btn.click(run_controlnet, [init_image, control_type, control_image, c_prompt, c_negative, c_steps, c_guidance, cn_scale, c_strength, c_seed], cn_out)

        with gr.Tab("3) Inpaint (foreground)"):
            base_img = gr.Image(type="pil", label="Base Image (use output from step 2 or 1)")
            mask_img = gr.Image(type="pil", label="Mask (white = edit region)")
            with gr.Row():
                i_prompt = gr.Textbox(label="Prompt", value="a small wooden boat in the foreground")
                i_negative = gr.Textbox(label="Negative", value="blurry, lowres, extra limbs")
            with gr.Row():
                i_steps = gr.Slider(10, 60, value=28, step=1, label="Steps")
                i_guidance = gr.Slider(1.0, 12.0, value=7.5, step=0.1, label="CFG")
                i_strength = gr.Slider(0.1, 1.0, value=0.85, step=0.05, label="Strength")
                i_seed = gr.Number(value=123, precision=0, label="Seed (-1 for random)")
            i_btn = gr.Button("Inpaint Foreground")
            inpaint_out = gr.Image(type="pil", label="Composited Output")
            i_btn.click(run_inpaint, [base_img, mask_img, i_prompt, i_negative, i_steps, i_guidance, i_strength, i_seed], inpaint_out)

        # Convenience wiring to pass images between tabs
        bg_out.change(lambda im: im, inputs=bg_out, outputs=init_image)
        cn_out.change(lambda im: im, inputs=cn_out, outputs=base_img)
        bg_out.change(lambda im: im, inputs=bg_out, outputs=base_img)

    return demo


In [14]:

lowvram = True
run_demo = True
run_example = False

if run_demo:
    demo = build_demo(lowvram=lowvram)
    demo.launch()
elif run_example:
    run_scripted_example(lowvram=lowvram)
else:
    print("Nothing to do. Set run_demo=True or run_example=True")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

...

unet/diffusion_pytorch_model.bin:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

vae/diffusion_pytorch_model.bin:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
You have disabled the safety checker for <class 'diffusers.pipelin

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3139c4de81f3b0db4d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
